In [134]:
from create_model import create_model
import torch
import torch.nn.utils.prune as prune
import numpy as np
import torch.nn as nn

from copy import deepcopy as deepcopy

In [112]:
X = np.zeros((10,12)) 

In [113]:
model, optimizer, loss_fn = create_model(X)

In [114]:
parameters = list(model.named_parameters())

In [118]:
t = parameters[0]

In [120]:
model.named_parameters

<bound method Module.named_parameters of MLP(
  (blocks): Sequential(
    (0): Block(
      (linear): Linear(in_features=12, out_features=4, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): Block(
      (linear): Linear(in_features=4, out_features=8, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (2): Block(
      (linear): Linear(in_features=8, out_features=4, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (head): Linear(in_features=4, out_features=1, bias=True)
)>

In [121]:
test =  nn.Conv2d(5, 3, 2)

In [133]:
list(test.named_parameters())[0][1].grad

In [ ]:
def save_before_step(theta, grad):
    

In [128]:
grad = t[1]
y = torch.zeros_like(x)
torch.where(grad != 0, x, y)

tensor([[ 0.2489,  0.1719,  0.2260, -0.1490,  0.1997,  0.2757, -0.0815, -0.0803,
         -0.1574, -0.2619,  0.0840,  0.0667],
        [ 0.2055,  0.0542,  0.1126,  0.0545,  0.1351, -0.1481,  0.1177,  0.0299,
          0.0538, -0.2755, -0.0453,  0.2380],
        [-0.0589,  0.2636,  0.1324,  0.2421,  0.1718,  0.2775,  0.1874,  0.1662,
          0.1811,  0.1737,  0.0603,  0.1692],
        [ 0.1241,  0.2064, -0.0653,  0.1505,  0.2868, -0.0273, -0.1765, -0.2356,
          0.2621,  0.0063,  0.2455, -0.2279]], grad_fn=<SWhereBackward>)

In [109]:
t[1]

Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], requires_grad=True)